In [ ]:
# https://www.bindingdb.org/bind/chemsearch/marvin/SDFdownload.jsp?all_download=yes
# BindingDB_BindingDB_Inhibition_2021m6.tsv.zip

from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools 
from rdkit.Chem.Draw import IPythonConsole  
import pandas as pd, sqlite3, math
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None) 

In [ ]:
con = sqlite3.connect('/scratch/xg590/scratch/BindingDB.db') #  :memory:
cur = con.cursor()

In [ ]:
df = pd.read_pickle('data/df_processed.pkl') 
df[['InChIKey','binder_smiles','unp_accessionid','unp_resnum','record_id']].to_sql('pdbcov',con,if_exists="replace",index=True)  

In [ ]:
pd.read_sql('select * from bindingdb limit 1', con)

In [ ]:
pd.read_sql('select * from pdbcov limit 1', con)

In [ ]:
Mapping = pd.read_sql_query('''SELECT pdbcov.adduct_id, bindingdb.react_id, unp_accessionid, binder_smiles, record_id
                               FROM pdbcov JOIN bindingdb
                               ON     pdbcov.unp_accessionid = bindingdb.unprotid
                                  AND pdbcov.InChIKey        = bindingdb.inchikey''' , con) 
# executed in 39.3s, finished 06:37:50 2021-05-21

In [ ]:
Mapping.drop_duplicates(['unp_accessionid','binder_smiles'])

In [ ]:
mapping = Mapping.sort_values('record_id')\
                 .drop_duplicates(['record_id'])[['record_id','react_id']]\
                 .set_index('record_id')\
                 .to_dict('dict')['react_id']
df['BindingDB_react_id']=df.apply(lambda r: str(mapping[r.record_id]) if r.record_id in mapping else math.nan,axis=1)
df.astype({'BindingDB_react_id'}).to_pickle('data/df_processed.pkl') 